### Track IPO Cleaned Notebook

In [43]:
import requests
import pprint
import requests
from requests.exceptions import HTTPError
from datetime import datetime as dt
import pandas as pd
import numpy as np
import timeit

def git_merge_commit_branch_simple(message):
    ! git add -A
    ! git commit -m f"{message}"
    ! git push origin eric

In [2]:
# test of generic base

for url in ["https://api.trackico.io/v1/icos/all/?page=1&page_size=200000&q="]:
    try:
        response = requests.get(url)

        # If the response was successful, no Exception will be raised
        response.raise_for_status()
    except HTTPError as http_err:
        print(f'HTTP error occurred: {http_err}')  # Python 3.6
    except Exception as err:
        print(f'Other error occurred: {err}')  # Python 3.6
    else:
        print('Success!')

Success!


In [4]:
type(response)

dict

In [5]:
#Christian modification

def get_Track_ICO_API():
    '''
    Function that calls 4,000 cryptos (ICO and IEO) and  from TrackICO. All features included.
    Tickers as index.
    Features = ['title', 'type', 'ticker', 'rating', 'status', 'country', 'platform', 'pre_ico_start', 'pre_ico_end', 'ico_start', 'ico_end']
    Returns df
    No 
    '''
    
    ICO_df = pd.DataFrame()

    for i in range(0,8):
        
        response = requests.get(f"https://api.trackico.io/v1/icos/all/?page=1&page_size=2&q=")
        response = response.json()
        
        print(f'Checking page{i}')
        
        for coin in response['results']:
            
            result = pd.DataFrame.from_dict(response['results'])

            ICO_df = pd.concat([result,ICO_df])
            
            print('Joining Dataframe')
            
    ICO_df.drop(['slug','id','category','profile_url','logo_url','short_description'], axis=1, inplace=True)
    ICO_df.reset_index(drop=True, inplace = True)
    ICO_df.set_index('title', inplace = True)
    return ICO_df

In [ ]:
# DO NOT RUN... takes about an hour to run this code. 2,000,000 results returned
ICO_API_df = get_Track_ICO_API()

In [ ]:
ICO_API_df

In [ ]:
len(ICO_API_df['ticker'].unique().tolist()) #474 unqiue cryptos/platforms

### Summary... only retrieving 474 unique ICO and IEO. 
### API must be either called differently or set up in this way on purpose
### ----------------------
### Original Code that calls the API, returns a df.

In [97]:
def get_Track_ICO_API():
    '''
    Function that calls 4,000 cryptos (ICO and IEO) and  from TrackICO. All features included.
    Tickers as index.
    Features = ['title', 'type', 'ticker', 'rating', 'status', 'country', 'platform', 'pre_ico_start', 'pre_ico_end', 'ico_start', 'ico_end']
    Returns df
    No 
    '''
    
    ICO_df = pd.DataFrame()

    for i in range(0,8):
        response = requests.get(f"https://api.trackico.io/v1/icos/all/?page=1&page_size=2000000&q=")

        result = pd.DataFrame.from_dict(response.json()['results'])

        ICO_df = pd.concat([result,ICO_df])
    
    ICO_df.drop(['slug','id','category','profile_url','logo_url','short_description'], axis=1, inplace=True)
    ICO_df.reset_index(drop=True, inplace = True)
    ICO_df.set_index('title', inplace = True)
    #ICO_df['Start'] = dt.strptime(ICO_df['ico_start'],"%Y-%m-%dT%H:%M:%SZ")
    #ICO_df['End'] = dt.strptime(ICO_df['ico_end'],"%Y-%m-%dT%H:%M:%SZ")
    #ICO_df['Pre_Start'] = dt.strptime(ICO_df['pre_ico_start'],"%Y-%m-%dT%H:%M:%SZ")
    #ICO_df['Pre_End'] = dt.strptime(ICO_df['pre_ico_end'],"%Y-%m-%dT%H:%M:%SZ")
    return ICO_df

In [98]:
%time track_ICO_API_original = get_Track_ICO_API()

Wall time: 5.23 s


In [99]:
track_ICO_API_original.head()

,country,ico_end,ico_start,platform,pre_ico_end,pre_ico_start,rating,status,ticker,type
title,,,,,,,,,,
DACX,United Kingdom,2020-01-10T00:00:00Z,2019-10-25T00:00:00Z,Ethereum,None,None,5.0,Ongoing,DACX,IEO
Gigajoule,Malta,2020-04-30T23:59:00Z,2020-01-01T12:00:00Z,Ethereum,2019-12-31T00:00:00Z,2019-11-01T00:00:00Z,4.4,Pre-Sale,GIGJ,STO
ProTravelcoin,Switzerland,None,None,Ethereum,2020-01-24T16:00:00Z,2019-12-13T16:00:00Z,3.9,Upcoming,PTR,ICO
Global Crypto Alliance,Malta,2019-11-28T08:00:00Z,2019-10-30T08:00:00Z,Ethereum,None,None,5.0,Ongoing,CALL,IEO
Statizex,Singapore,2019-11-24T00:00:00Z,2019-10-10T00:00:00Z,Ethereum,None,None,4.1,Ongoing,STAT,IEO


In [18]:
track_ICO_API_original.shape #says 4000 but its not unique

(4000, 10)

### Analyzing issues. Says 4000... the API call counts 4000... but its not.

In [19]:
list_of_lists = []

for ticker in track_ICO_API_original.index.unique().tolist():
    
    if len(track_ICO_API_original.index[track_ICO_API_original['ticker'] == ticker].tolist()) != 0:
        
        list = track_ICO_API_original.index[track_ICO_API_original['ticker'] == ticker].tolist()
    
    else:
        continue
    
    list_of_lists.append(list)
    
select_indicies_list = []

for ticker in track_ICO_API_original.index.unique().tolist():
    
    select_indices = np.where((track_ICO_API_original["ticker"] == ticker).tolist())
    
    select_indicies_list.append(select_indices)

In [26]:
list_of_lists

[['DACX', 'DACX', 'DACX', 'DACX', 'DACX', 'DACX', 'DACX', 'DACX'],
 ['EZ365', 'EZ365', 'EZ365', 'EZ365', 'EZ365', 'EZ365', 'EZ365', 'EZ365'],
 ['DIAM', 'DIAM', 'DIAM', 'DIAM', 'DIAM', 'DIAM', 'DIAM', 'DIAM'],
 ['EXBEGA',
  'EXBEGA',
  'EXBEGA',
  'EXBEGA',
  'EXBEGA',
  'EXBEGA',
  'EXBEGA',
  'EXBEGA'],
 ['PBET', 'PBET', 'PBET', 'PBET', 'PBET', 'PBET', 'PBET', 'PBET'],
 ['NGS', 'NGS', 'NGS', 'NGS', 'NGS', 'NGS', 'NGS', 'NGS'],
 ['WOX', 'WOX', 'WOX', 'WOX', 'WOX', 'WOX', 'WOX', 'WOX'],
 ['BINCOIN',
  'BINCOIN',
  'BINCOIN',
  'BINCOIN',
  'BINCOIN',
  'BINCOIN',
  'BINCOIN',
  'BINCOIN'],
 ['LXDX', 'LXDX', 'LXDX', 'LXDX', 'LXDX', 'LXDX', 'LXDX', 'LXDX'],
 ['edeXa', 'edeXa', 'edeXa', 'edeXa', 'edeXa', 'edeXa', 'edeXa', 'edeXa'],
 ['Pahoo', 'Pahoo', 'Pahoo', 'Pahoo', 'Pahoo', 'Pahoo', 'Pahoo', 'Pahoo'],
 ['Viacash',
  'Viacash',
  'Viacash',
  'Viacash',
  'Viacash',
  'Viacash',
  'Viacash',
  'Viacash'],
 ['WIBX', 'WIBX', 'WIBX', 'WIBX', 'WIBX', 'WIBX', 'WIBX', 'WIBX'],
 ['WOWX', 'WOWX

In [27]:
select_indicies_list #not all being duplicated and they are set intervals appart.... 500 apart

[(array([   0,  500, 1000, 1500, 2000, 2500, 3000, 3500], dtype=int64),),
 (array([], dtype=int64),),
 (array([], dtype=int64),),
 (array([], dtype=int64),),
 (array([], dtype=int64),),
 (array([], dtype=int64),),
 (array([], dtype=int64),),
 (array([], dtype=int64),),
 (array([], dtype=int64),),
 (array([], dtype=int64),),
 (array([], dtype=int64),),
 (array([], dtype=int64),),
 (array([], dtype=int64),),
 (array([], dtype=int64),),
 (array([], dtype=int64),),
 (array([], dtype=int64),),
 (array([], dtype=int64),),
 (array([], dtype=int64),),
 (array([], dtype=int64),),
 (array([], dtype=int64),),
 (array([], dtype=int64),),
 (array([], dtype=int64),),
 (array([], dtype=int64),),
 (array([], dtype=int64),),
 (array([  24,  524, 1024, 1524, 2024, 2524, 3024, 3524], dtype=int64),),
 (array([], dtype=int64),),
 (array([], dtype=int64),),
 (array([], dtype=int64),),
 (array([], dtype=int64),),
 (array([], dtype=int64),),
 (array([], dtype=int64),),
 (array([], dtype=int64),),
 (array([], 

In [20]:
select_indices = np.where(track_ICO_API_original["ticker"] == 'EZ365')
select_indices

(array([  24,  524, 1024, 1524, 2024, 2524, 3024, 3524], dtype=int64),)

In [23]:
#manual pull of array for analysis
EZ365 = [24,  524, 1024, 1524, 2024, 2524, 3024, 3524]

df2 = pd.DataFrame()

for index in EZ365:
    df = track_ICO_API_original.reset_index().iloc[[24]]
    df2 = pd.concat([df,df2])

In [24]:
df2 #just repeating

,title,country,ico_end,ico_start,platform,pre_ico_end,pre_ico_start,rating,status,ticker,type
24,EZ365,None,2019-11-01T01:59:00Z,2019-09-23T16:00:00Z,Ethereum,2019-09-22T15:59:00Z,2019-09-05T16:00:00Z,4.6,Closed,EZ365,IEO
24,EZ365,None,2019-11-01T01:59:00Z,2019-09-23T16:00:00Z,Ethereum,2019-09-22T15:59:00Z,2019-09-05T16:00:00Z,4.6,Closed,EZ365,IEO
24,EZ365,None,2019-11-01T01:59:00Z,2019-09-23T16:00:00Z,Ethereum,2019-09-22T15:59:00Z,2019-09-05T16:00:00Z,4.6,Closed,EZ365,IEO
24,EZ365,None,2019-11-01T01:59:00Z,2019-09-23T16:00:00Z,Ethereum,2019-09-22T15:59:00Z,2019-09-05T16:00:00Z,4.6,Closed,EZ365,IEO
24,EZ365,None,2019-11-01T01:59:00Z,2019-09-23T16:00:00Z,Ethereum,2019-09-22T15:59:00Z,2019-09-05T16:00:00Z,4.6,Closed,EZ365,IEO
24,EZ365,None,2019-11-01T01:59:00Z,2019-09-23T16:00:00Z,Ethereum,2019-09-22T15:59:00Z,2019-09-05T16:00:00Z,4.6,Closed,EZ365,IEO
24,EZ365,None,2019-11-01T01:59:00Z,2019-09-23T16:00:00Z,Ethereum,2019-09-22T15:59:00Z,2019-09-05T16:00:00Z,4.6,Closed,EZ365,IEO
24,EZ365,None,2019-11-01T01:59:00Z,2019-09-23T16:00:00Z,Ethereum,2019-09-22T15:59:00Z,2019-09-05T16:00:00Z,4.6,Closed,EZ365,IEO


In [25]:
len(track_ICO_API_original.index.unique().tolist()) #what?

500

### rest of code dropped... checking datatimes and such since this is not working for the momemnt
### -----------
### dropping duplicates and creating a df

In [100]:
track_ICO_API_original.head()

,country,ico_end,ico_start,platform,pre_ico_end,pre_ico_start,rating,status,ticker,type
title,,,,,,,,,,
DACX,United Kingdom,2020-01-10T00:00:00Z,2019-10-25T00:00:00Z,Ethereum,None,None,5.0,Ongoing,DACX,IEO
Gigajoule,Malta,2020-04-30T23:59:00Z,2020-01-01T12:00:00Z,Ethereum,2019-12-31T00:00:00Z,2019-11-01T00:00:00Z,4.4,Pre-Sale,GIGJ,STO
ProTravelcoin,Switzerland,None,None,Ethereum,2020-01-24T16:00:00Z,2019-12-13T16:00:00Z,3.9,Upcoming,PTR,ICO
Global Crypto Alliance,Malta,2019-11-28T08:00:00Z,2019-10-30T08:00:00Z,Ethereum,None,None,5.0,Ongoing,CALL,IEO
Statizex,Singapore,2019-11-24T00:00:00Z,2019-10-10T00:00:00Z,Ethereum,None,None,4.1,Ongoing,STAT,IEO


In [101]:
track_ICO_API_original.drop_duplicates().shape

(500, 10)

In [102]:
track_ICO_API_original.drop_duplicates(inplace=True)

In [103]:
track_ICO_API_original.head()

,country,ico_end,ico_start,platform,pre_ico_end,pre_ico_start,rating,status,ticker,type
title,,,,,,,,,,
DACX,United Kingdom,2020-01-10T00:00:00Z,2019-10-25T00:00:00Z,Ethereum,None,None,5.0,Ongoing,DACX,IEO
Gigajoule,Malta,2020-04-30T23:59:00Z,2020-01-01T12:00:00Z,Ethereum,2019-12-31T00:00:00Z,2019-11-01T00:00:00Z,4.4,Pre-Sale,GIGJ,STO
ProTravelcoin,Switzerland,None,None,Ethereum,2020-01-24T16:00:00Z,2019-12-13T16:00:00Z,3.9,Upcoming,PTR,ICO
Global Crypto Alliance,Malta,2019-11-28T08:00:00Z,2019-10-30T08:00:00Z,Ethereum,None,None,5.0,Ongoing,CALL,IEO
Statizex,Singapore,2019-11-24T00:00:00Z,2019-10-10T00:00:00Z,Ethereum,None,None,4.1,Ongoing,STAT,IEO


In [104]:
list_of_lists = []

for ticker in track_ICO_API_original.index.unique().tolist():
    
    if len(track_ICO_API_original.index[track_ICO_API_original['ticker'] == ticker].tolist()) != 0:
        
        list = track_ICO_API_original.index[track_ICO_API_original['ticker'] == ticker].tolist()
    
    else:
        continue
    
    list_of_lists.append(list)
    
select_indicies_list = []

for ticker in track_ICO_API_original.index.unique().tolist():
    
    select_indices = np.where((track_ICO_API_original["ticker"] == ticker).tolist())
    
    select_indicies_list.append(select_indices)

In [84]:
len(list_of_lists) #31 still 

31

In [83]:
select_indicies_list #still a few...

[(array([0], dtype=int64),),
 (array([], dtype=int64),),
 (array([], dtype=int64),),
 (array([], dtype=int64),),
 (array([], dtype=int64),),
 (array([], dtype=int64),),
 (array([], dtype=int64),),
 (array([], dtype=int64),),
 (array([], dtype=int64),),
 (array([], dtype=int64),),
 (array([], dtype=int64),),
 (array([], dtype=int64),),
 (array([], dtype=int64),),
 (array([], dtype=int64),),
 (array([], dtype=int64),),
 (array([], dtype=int64),),
 (array([], dtype=int64),),
 (array([], dtype=int64),),
 (array([], dtype=int64),),
 (array([], dtype=int64),),
 (array([], dtype=int64),),
 (array([], dtype=int64),),
 (array([], dtype=int64),),
 (array([], dtype=int64),),
 (array([24], dtype=int64),),
 (array([], dtype=int64),),
 (array([], dtype=int64),),
 (array([], dtype=int64),),
 (array([], dtype=int64),),
 (array([], dtype=int64),),
 (array([], dtype=int64),),
 (array([], dtype=int64),),
 (array([], dtype=int64),),
 (array([], dtype=int64),),
 (array([], dtype=int64),),
 (array([], dtype

In [89]:
#track_ICO_API_original_ = track_ICO_API_original.reset_index().drop_duplicates(subset='title', keep='first')

In [92]:
track_ICO_API_original_.shape #still 500?

(500, 13)